In [2]:
from google.colab import drive
import os
import zipfile

# Kết nối Google Drive
drive.mount('/content/drive')

# Đường dẫn tới thư mục dự án trên Drive
project_dir = "/content/drive/MyDrive/Project_Gki"
print("Nội dung trong Project_Gki:")
print(os.listdir(project_dir))  # Liệt kê file/thư mục trong Project_Gki

# Đường dẫn tới file zip
zip_path = os.path.join(project_dir, "StanfordCars.zip")
print("Đường dẫn zip:", zip_path)
print("Tồn tại file zip?", os.path.exists(zip_path))

# Giải nén file zip
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/stanford_cars")
    print("Đã giải nén StanfordCars.zip")
else:
    print("File zip không tồn tại!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Nội dung trong Project_Gki:
['StanfordCars.zip', 'Copy of Gki_Stanford_Cars.ipynb']
Đường dẫn zip: /content/drive/MyDrive/Project_Gki/StanfordCars.zip
Tồn tại file zip? True
Đã giải nén StanfordCars.zip


In [4]:
import os
import json
import random
from PIL import Image

# Đường dẫn đến thư mục chứa ảnh
IMAGE_DIR = "/content/stanford_cars/cars_train/cars_train"
OUTPUT_FILE = "/content/questions_answers.json"

# Các loại câu hỏi và câu trả lời mẫu
question_templates = [
    "Có bao nhiêu xe trong ảnh?",
    "Màu sắc của xe là gì?",
    "Đây là loại xe gì?",
    "Xe này được sản xuất bởi hãng nào?",
    "Xe trong ảnh đang ở đâu?"
]

color_answers = ["đỏ", "xanh dương", "xanh lá", "đen", "trắng", "bạc", "vàng", "cam", "xám"]
car_types = ["sedan", "SUV", "xe thể thao", "xe bán tải", "xe tải", "xe sang"]
car_brands = ["Toyota", "Honda", "BMW", "Mercedes", "Ford", "Audi", "Hyundai", "Kia"]
locations = ["trên đường phố", "trong bãi đỗ xe", "trước tòa nhà", "trong garage", "bên đường"]

def generate_qa_pairs(image_path):
    """Tạo câu hỏi và câu trả lời cho một hình ảnh"""
    filename = os.path.basename(image_path)
    qa_pairs = []

    # Câu hỏi về số lượng (giả định mỗi ảnh chỉ có một chiếc xe)
    qa_pairs.append({
        "image": filename,
        "question": "Có bao nhiêu xe trong ảnh?",
        "answer": "Có một chiếc xe trong ảnh."
    })

    # Câu hỏi về màu sắc
    color = random.choice(color_answers)
    qa_pairs.append({
        "image": filename,
        "question": "Màu sắc của xe là gì?",
        "answer": f"Chiếc xe có màu {color}."
    })

    # Câu hỏi về loại xe
    car_type = random.choice(car_types)
    qa_pairs.append({
        "image": filename,
        "question": "Đây là loại xe gì?",
        "answer": f"Đây là một chiếc {car_type}."
    })

    # Câu hỏi về thương hiệu
    brand = random.choice(car_brands)
    qa_pairs.append({
        "image": filename,
        "question": "Xe này được sản xuất bởi hãng nào?",
        "answer": f"Đây là xe {brand}."
    })

    # Câu hỏi về vị trí
    location = random.choice(locations)
    qa_pairs.append({
        "image": filename,
        "question": "Xe trong ảnh đang ở đâu?",
        "answer": f"Chiếc xe đang ở {location}."
    })

    return qa_pairs

def create_dataset(image_dir, output_file):
    """Tạo bộ dữ liệu từ thư mục hình ảnh"""
    all_qa_pairs = []

    # Kiểm tra thư mục tồn tại
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
        print(f"Đã tạo thư mục {image_dir}. Vui lòng thêm các hình ảnh xe vào thư mục này.")
        return

    # Lấy tất cả các file hình ảnh
    image_files = [f for f in os.listdir(image_dir)
                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print(f"Không tìm thấy hình ảnh nào trong thư mục {image_dir}")
        return

    print(f"Tạo bộ dữ liệu từ {len(image_files)} hình ảnh...")

    # Tạo câu hỏi và câu trả lời cho mỗi hình ảnh
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        qa_pairs = generate_qa_pairs(image_path)
        all_qa_pairs.extend(qa_pairs)

    # Lưu vào file JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_qa_pairs, f, ensure_ascii=False, indent=2)

    print(f"Đã tạo {len(all_qa_pairs)} cặp câu hỏi-trả lời trong file {output_file}")
    return all_qa_pairs

if __name__ == "__main__":
    create_dataset(IMAGE_DIR, OUTPUT_FILE)

Tạo bộ dữ liệu từ 8144 hình ảnh...
Đã tạo 40720 cặp câu hỏi-trả lời trong file /content/questions_answers.json


In [8]:
import torch
import torch.nn as nn
from torchvision import models

# CNN Encoder (với lựa chọn pretrained hoặc train từ đầu)
class CNNEncoder(nn.Module):
    def __init__(self, embed_size=512, pretrained=True):
        super(CNNEncoder, self).__init__()
        if pretrained:
            cnn = models.resnet50(pretrained=True)
            # Đóng băng các tham số nếu sử dụng pretrained
            for param in cnn.parameters():
                param.requires_grad = False
        else:
            # Train từ đầu
            cnn = models.resnet50(pretrained=False)

        self.cnn = nn.Sequential(*list(cnn.children())[:-1])
        self.fc = nn.Linear(2048, embed_size)

    def forward(self, images):
        features = self.cnn(images)
        features = features.view(features.size(0), -1)
        features = self.fc(features)
        return features

# Bộ mã hóa câu hỏi
class QuestionEncoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(QuestionEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, questions):
        embeddings = self.embedding(questions)
        outputs, (hidden, cell) = self.lstm(embeddings)
        features = self.fc(hidden.squeeze(0))
        return outputs, features

# LSTM Decoder với Attention
class AttentionLSTMDecoder(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size):
        super(AttentionLSTMDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.attention = nn.Linear(hidden_size * 2, hidden_size)
        self.attention_combine = nn.Linear(hidden_size * 2 + embed_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, img_features, q_features, captions):
        batch_size = captions.size(0)
        embeddings = self.embedding(captions)

        # Kết hợp đặc trưng câu hỏi với đặc trưng hình ảnh
        combined_features = torch.cat((img_features, q_features), dim=1)
        context = self.attention(combined_features)

        # Lặp lại ngữ cảnh cho mỗi từ trong chuỗi
        context = context.unsqueeze(1).repeat(1, embeddings.size(1), 1)

        # Nối embeddings và ngữ cảnh
        lstm_input = torch.cat((embeddings, context), dim=2)

        # LSTM và đầu ra
        hiddens, _ = self.lstm(lstm_input)
        outputs = self.fc(hiddens)
        return outputs

# LSTM Decoder không có Attention
class BasicLSTMDecoder(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size):
        super(BasicLSTMDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size + hidden_size * 2, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, img_features, q_features, captions):
        embeddings = self.embedding(captions)

        # Kết hợp đặc trưng
        combined_features = torch.cat((img_features, q_features), dim=1)

        # Lặp lại cho mỗi từ
        features = combined_features.unsqueeze(1).repeat(1, embeddings.size(1), 1)
        inputs = torch.cat((features, embeddings), dim=2)

        # LSTM và đầu ra
        hiddens, _ = self.lstm(inputs)
        outputs = self.fc(hiddens)
        return outputs

# Mô hình VQA hoàn chỉnh
class VQAModel(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, use_pretrained=True, use_attention=True):
        super(VQAModel, self).__init__()
        self.cnn = CNNEncoder(embed_size, pretrained=use_pretrained)
        self.question_encoder = QuestionEncoder(vocab_size, embed_size, hidden_size)

        if use_attention:
            self.decoder = AttentionLSTMDecoder(embed_size, hidden_size, vocab_size)
        else:
            self.decoder = BasicLSTMDecoder(embed_size, hidden_size, vocab_size)

        self.use_attention = use_attention

    def forward(self, images, questions, answers=None):
        img_features = self.cnn(images)
        q_outputs, q_features = self.question_encoder(questions)

        if answers is not None:
            # Chế độ huấn luyện - teacher forcing
            outputs = self.decoder(img_features, q_features, answers[:, :-1])
            return outputs
        else:
            # Chế độ suy luận
            return img_features, q_features

In [9]:
import os
import json
import torch
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Transform ảnh
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def load_image(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
        return transform(image)
    except Exception as e:
        print(f"Lỗi khi tải hình ảnh {image_path}: {e}")
        # Trả về tensor trống nếu không tải được hình ảnh
        return torch.zeros(3, 224, 224)

# Xây dựng từ điển
def build_vocab(data):
    vocab = set()
    for item in data:
        vocab.update(item['question'].split())
        vocab.update(item['answer'].split())
    vocab = ['<PAD>', '<UNK>', '<START>', '<END>'] + list(vocab)
    return {word: idx for idx, word in enumerate(vocab)}, {idx: word for idx, word in enumerate(vocab)}

# Dataset
class QADataset(Dataset):
    def __init__(self, data, word2idx, image_dir):
        self.data = data
        self.word2idx = word2idx
        self.image_dir = image_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image_path = os.path.join(self.image_dir, item['image'])
        image = load_image(image_path)

        # Chuyển câu hỏi thành token
        question = [self.word2idx.get(word, self.word2idx['<UNK>']) for word in item['question'].split()]

        # Chuyển câu trả lời thành token
        answer = [self.word2idx['<START>']] + [self.word2idx.get(word, self.word2idx['<UNK>'])
                                              for word in item['answer'].split()] + [self.word2idx['<END>']]

        return image, torch.tensor(question), torch.tensor(answer)

def collate_fn(batch):
    images, questions, answers = zip(*batch)
    images = torch.stack(images)
    questions = nn.utils.rnn.pad_sequence(questions, batch_first=True, padding_value=0)
    answers = nn.utils.rnn.pad_sequence(answers, batch_first=True, padding_value=0)
    return images, questions, answers

def prepare_dataloaders(data_file, image_dir, batch_size=32):
    # Đọc dữ liệu
    try:
        with open(data_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"File dữ liệu {data_file} không tồn tại!")
        return None, None, None, None, None

    # Xây dựng từ điển
    word2idx, idx2word = build_vocab(data)
    vocab_size = len(word2idx)

    # Chia dữ liệu
    train_size = int(0.8 * len(data))
    val_size = int(0.1 * len(data))

    train_data = data[:train_size]
    val_data = data[train_size:train_size + val_size]
    test_data = data[train_size + val_size:]

    # Tạo datasets
    train_dataset = QADataset(train_data, word2idx, image_dir)
    val_dataset = QADataset(val_data, word2idx, image_dir)
    test_dataset = QADataset(test_data, word2idx, image_dir)

    # Tạo dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        collate_fn=collate_fn
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=1,  # Để dễ xử lý, batch_size cho test_loader là 1
        collate_fn=collate_fn
    )

    return train_loader, val_loader, test_loader, word2idx, idx2word

In [10]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from model import VQAModel
from dataset import prepare_dataloaders, load_image

# Cấu hình
CONFIG = {
    'embed_size': 256,
    'hidden_size': 512,
    'batch_size': 32,
    'epochs': 10,
    'learning_rate': 0.001,
    'image_dir': '/content/stanford_cars/cars_train',
    'data_file': '/content/questions_answers.json',
    'save_dir': '/content/models',
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

# Tạo thư mục lưu mô hình
if not os.path.exists(CONFIG['save_dir']):
    os.makedirs(CONFIG['save_dir'])

def train_epoch(model, train_loader, criterion, optimizer, device):
    """Huấn luyện mô hình qua 1 epoch"""
    model.train()
    total_loss = 0.0
    start_time = time.time()

    for i, (images, questions, answers) in enumerate(train_loader):
        images = images.to(device)
        questions = questions.to(device)
        answers = answers.to(device)

        # Forward pass
        outputs = model(images, questions, answers)

        # Tính loss (bỏ qua padding)
        loss = criterion(outputs.reshape(-1, outputs.size(-1)), answers[:, 1:].reshape(-1))

        # Backward pass và optimize
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)  # Gradient clipping
        optimizer.step()

        total_loss += loss.item()

        # In tiến trình
        if (i + 1) % 10 == 0:
            print(f"Batch {i+1}/{len(train_loader)}, Loss: {loss.item():.4f}, "
                  f"Time: {time.time() - start_time:.2f}s")
            start_time = time.time()

    return total_loss / len(train_loader)

def validate(model, val_loader, criterion, device):
    """Đánh giá mô hình trên tập validation"""
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for images, questions, answers in val_loader:
            images = images.to(device)
            questions = questions.to(device)
            answers = answers.to(device)

            # Forward pass
            outputs = model(images, questions, answers)

            # Tính loss
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), answers[:, 1:].reshape(-1))
            total_loss += loss.item()

    return total_loss / len(val_loader)

def generate_answer(model, image, question, word2idx, idx2word, device, max_len=20):
    """Sinh câu trả lời từ mô hình"""
    model.eval()
    with torch.no_grad():
        # Chuẩn bị đầu vào
        image = image.unsqueeze(0).to(device)
        question = torch.tensor([word2idx.get(word, word2idx['<UNK>'])
                                for word in question.split()]).unsqueeze(0).to(device)

        # Lấy đặc trưng
        img_features, q_features = model(image, question)

        # Sinh câu trả lời
        answer = [word2idx['<START>']]
        for i in range(max_len):
            # Chuyển đổi chuỗi hiện tại thành tensor
            current_answer = torch.tensor([answer]).to(device)

            # Dự đoán từ tiếp theo
            output = model.decoder(img_features, q_features, current_answer)

            # Lấy từ có khả năng cao nhất tiếp theo
            _, next_word_idx = output[:, -1].max(1)
            next_word_idx = next_word_idx.item()

            # Thêm vào chuỗi câu trả lời
            answer.append(next_word_idx)

            # Dừng nếu gặp token <END>
            if next_word_idx == word2idx['<END>']:
                break

        # Chuyển đổi chỉ số thành từ (bỏ qua <START> và <END>)
        words = [idx2word[idx] for idx in answer[1:-1] if idx in idx2word]
        return ' '.join(words)

def train_model():
    """Huấn luyện các biến thể của mô hình và so sánh hiệu suất"""
    print(f"Sử dụng thiết bị: {CONFIG['device']}")

    # Chuẩn bị dữ liệu
    train_loader, val_loader, test_loader, word2idx, idx2word = prepare_dataloaders(
        CONFIG['data_file'],
        CONFIG['image_dir'],
        CONFIG['batch_size']
    )

    if train_loader is None:
        print("Không thể tải dữ liệu. Vui lòng chạy data_generation.py trước.")
        return

    vocab_size = len(word2idx)
    print(f"Kích thước từ điển: {vocab_size}")

    # Định nghĩa 4 mô hình khác nhau để so sánh
    models = {
        "Pretrained_CNN_with_Attention": VQAModel(
            embed_size=CONFIG['embed_size'],
            hidden_size=CONFIG['hidden_size'],
            vocab_size=vocab_size,
            use_pretrained=True,
            use_attention=True
        ),
        "Pretrained_CNN_without_Attention": VQAModel(
            embed_size=CONFIG['embed_size'],
            hidden_size=CONFIG['hidden_size'],
            vocab_size=vocab_size,
            use_pretrained=True,
            use_attention=False
        ),
        "From_Scratch_CNN_with_Attention": VQAModel(
            embed_size=CONFIG['embed_size'],
            hidden_size=CONFIG['hidden_size'],
            vocab_size=vocab_size,
            use_pretrained=False,
            use_attention=True
        ),
        "From_Scratch_CNN_without_Attention": VQAModel(
            embed_size=CONFIG['embed_size'],
            hidden_size=CONFIG['hidden_size'],
            vocab_size=vocab_size,
            use_pretrained=False,
            use_attention=False
        )
    }

    results = {}

    # Huấn luyện từng mô hình
    for model_name, model in models.items():
        print(f"\n{'='*50}")
        print(f"Đang huấn luyện {model_name}...")
        print(f"{'='*50}")

        model = model.to(CONFIG['device'])

        criterion = nn.CrossEntropyLoss(ignore_index=word2idx['<PAD>'])
        optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'])
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

        train_losses = []
        val_losses = []

        for epoch in range(CONFIG['epochs']):
            print(f"\nEpoch {epoch+1}/{CONFIG['epochs']}")
            print("-" * 30)

            # Huấn luyện
            train_loss = train_epoch(model, train_loader, criterion, optimizer, CONFIG['device'])
            train_losses.append(train_loss)

            # Kiểm tra
            val_loss = validate(model, val_loader, criterion, CONFIG['device'])
            val_losses.append(val_loss)

            # Điều chỉnh learning rate
            scheduler.step(val_loss)

            print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Lưu kết quả
        results[model_name] = {
            'model': model,
            'train_losses': train_losses,
            'val_losses': val_losses
        }

        # Lưu mô hình
        torch.save(model.state_dict(), os.path.join(CONFIG['save_dir'], f"{model_name}.pth"))
        print(f"Đã lưu mô hình tại {os.path.join(CONFIG['save_dir'], model_name)}.pth")

    # Vẽ đồ thị so sánh
    plt.figure(figsize=(12, 8))
    for model_name, result in results.items():
        plt.plot(result['val_losses'], label=f"{model_name}")

    plt.title('So sánh Loss đánh giá')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(CONFIG['save_dir'], 'model_comparison.png'))
    print(f"Đã lưu biểu đồ so sánh tại {os.path.join(CONFIG['save_dir'], 'model_comparison.png')}")

    # Đánh giá trên tập kiểm tra
    print("\nĐánh giá mô hình trên tập kiểm tra...")

    for model_name, result in results.items():
        model = result['model']
        model.eval()

        print(f"\nMô hình: {model_name}")
        for i, (image, question, answer) in enumerate(test_loader):
            if i >= 3:  # Chỉ hiển thị 3 ví dụ đầu tiên
                break

            image = image[0]
            question_text = ' '.join([idx2word[idx.item()] for idx in question[0] if idx.item() in idx2word])
            answer_text = ' '.join([idx2word[idx.item()] for idx in answer[0][1:-1] if idx.item() in idx2word])

            generated = generate_answer(model, image, question_text, word2idx, idx2word, CONFIG['device'])

            print(f"Câu hỏi: {question_text}")
            print(f"Câu trả lời thực: {answer_text}")
            print(f"Câu trả lời sinh: {generated}\n")

if __name__ == "__main__":
    train_model()

ModuleNotFoundError: No module named 'model'